# <p style="text-align:center">Lecture 3. E-Cell4の構造</p>

<p style="text-align:center;font-size:150%;line-height:150%">海津一成</p>

In [3]:
%matplotlib inline
from ecell4 import *

## E-Cell4の基本要素

E-Cell4を使う上で理解しなければならない3つの要素として`Model`、`World`、`Simulator`があり、計算における異なる概念をあらわしている。

- `Model`: いわゆるモデル。分子の特徴量（拡散定数、大きさなど）や反応を記述したもの。具体的には`NetworkModel`を使う。

- `World`: ある時刻における状態をあらわすもの。分子の数や位置、空間の表現法など。使う`World`はアルゴリズムによって異なる. 例えば、`ode.ODEWorld`みたいな感じ。

- Simulator: 時間発展に関する計算手法をまとめたもの。ソルバーとも言う。例えば、`ode.ODESimulator`みたいな感じ。

## E-Cell4におけるModelの基本

Modelは扱う問題を記述したもので、大きく2つの要素によって構成されている。

- `Species`: すなわち分子種。モデルに登場する実体をあらわす。
- `ReactionRule`: すなわち反応則。分子種（`Species`）間の相互作用をあらわす。

E-Cell4では主に化学反応を扱うので、それを例に言えば

- `Species`: 基質や酵素などどんな種類の分子が存在し、それらがどのような性質 (分子の大きさなど)を持つかをあらわす。
- `ReactionRule`: 結合やかい離などそれら分子間にどのような反応があるかをあらわす。

## 分子種 `Species`

E-Cell4を使って`Species`を作るには

In [4]:
A = Species('A')
B = Species('B')

とすれば良い。

それぞれ、AとBという名前の分子種をあらわす。1個の特定の分子ではなく、そういう種類の分子があることを意味している。

`Species`には任意の属性を持たせることで、どのような性質を持つのかを指定できる。

In [5]:
A = Species("A")
A.set_attribute("radius", "0.005")
A.set_attribute("D", "1")
A.set_attribute("location", "cytoplasm")

1つ目に与える引数が属性名で、2つ目がその値で、どちらも文字列でなければならない。

属性のうち特に分子径、拡散係数、属する場所は頻繁に用いられるので簡単に

In [6]:
A = Species("A", "0.005", "1", "cytoplasm")  # XXX: serial, radius, D, location

でまとめて与えることができる。

与えたSpeciesの属性を知りたいときは

In [7]:
print(A.serial())
print(A.get_attribute("radius"))
print(A.get_attribute("D"))
print(A.get_attribute("location"))

A
0.005
1
cytoplasm


とすれば良い。

## 反応則 `ReactionRule`

`Species`ができたところで今度は反応を作ってみる。

反応はおおざっぱに言って、何かの分子種が何かの分子種になること。

前者をReactants、後者をProductsと呼ぶ。あとはその反応の速さ（速度定数）を数値で与えれば良い。

In [8]:
rr = ReactionRule()
rr.add_reactant(Species("A"))
rr.add_product(Species("B"))
rr.set_k(1.0)

これで分子種`A`が分子種`B`に変る反応を作れた。ここでは反応を定義しているので、`Species`に属性値はなくても良い。

確認してみる。与えられた`ReactionRule`がどのようなものかを簡単に知りたい場合は`as_string`関数で見られる。

In [10]:
print(rr.as_string())

A>B|1


結合もこれと同じに記述できる。`A`と`B`の結合によって`C`ができる。

In [11]:
rr = ReactionRule()
rr.add_reactant(Species("A"))
rr.add_reactant(Species("B"))
rr.add_product(Species("C"))
rr.set_k(1.0)

print(rr.as_string())

A+B>C|1


より読み手に伝わりやすい書き方として以下の便利関数も用意されている。

In [13]:
rr1 = create_unimolecular_reaction_rule(
    Species("A"), Species("B"), 1.0)
rr2 = create_binding_reaction_rule(
    Species("A"), Species("B"), Species("C"), 1.0)
rr3 = create_unbinding_reaction_rule(
    Species("C"), Species("A"), Species("B"), 1.0)